In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from google.colab import data_table
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from textblob import TextBlob

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/ML datasets/emotion-dataset(updated-final).csv")
data_table.DataTable(df.head())

,Emotion,Text
0,neutral,Why ?
1,joy,Sage Act upgrade on my to do list for tommorow.
2,sadness,ON THE WAY TO MY HOMEGIRL BABY FUNERAL!!! MAN ...
3,joy,Such an eye ! The true hazel eye-and so brill...
4,joy,@Iluvmiasantos ugh babe.. hugggzzz for u .! b...


In [ ]:
df['Emotion'].value_counts()

sadness     16553
joy         16407
fear         7347
anger        6566
love         5146
surprise     4634
neutral      2254
disgust       856
shame         146
Name: Emotion, dtype: int64

In [ ]:
df = df[df.Emotion != 'disgust']
df = df[df.Emotion != 'surprise']
df = df[df.Emotion != 'shame']
# df = df[df.Emotion != 'fear']
# df = df[df.Emotion != 'anger']
# df = df[df.Emotion != 'love']
# df = df[df.Emotion != 'neutral']

In [ ]:
!pip install neattext
import neattext.functions as nfx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
nltk.download("stopwords")
nltk.download("wordnet")

def clean_sentence(sentence):
    # Remove special characters and numbers
    sentence = re.sub(r'[^a-zA-Z]', ' ', sentence)
    # Convert to lowercase
    sentence = sentence.lower()
    # Remove URLs
    sentence = re.sub(r'http\S+', '', sentence)
    # Split into words
    sentence = sentence.split()
    # Remove stopwords
    sentence = [word for word in sentence if not word in set(stopwords.words('english'))]
    # Perform lemmatization
    lemmatizer = WordNetLemmatizer()
    sentence = [lemmatizer.lemmatize(word) for word in sentence]
    # Join the words back together
    sentence = ' '.join(sentence)
    return sentence


df['Clean_Text'] = df['Text'].apply(nfx.remove_punctuations)
df['Clean_Text'] = df['Clean_Text'].apply(nfx.remove_stopwords)
df['Clean_Text'] = df['Clean_Text'].apply(nfx.remove_userhandles)
df['Clean_Text'] = df['Text'].apply(lambda x: clean_sentence(x))


data_table.DataTable(df[['Text','Clean_Text']].head())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Text,Clean_Text
0,Why ?,
1,Sage Act upgrade on my to do list for tommorow.,sage act upgrade list tommorow
2,ON THE WAY TO MY HOMEGIRL BABY FUNERAL!!! MAN ...,way homegirl baby funeral man hate funeral rea...
3,Such an eye ! The true hazel eye-and so brill...,eye true hazel eye brilliant regular feature o...
4,@Iluvmiasantos ugh babe.. hugggzzz for u .! b...,iluvmiasantos ugh babe hugggzzz u babe naamaze...


In [ ]:
df.isnull().sum()

Emotion       0
Text          0
Clean_Text    0
dtype: int64

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Load data
data = df

# Split data into training, validation, and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(data['Clean_Text'], data['Emotion'], test_size=0.2, random_state=42)
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

# Vectorize text data using CountVectorizer
vectorizer = CountVectorizer()
train_features = vectorizer.fit_transform(train_data)
val_features = vectorizer.transform(val_data)
test_features = vectorizer.transform(test_data)

# Train a Naive Bayes model
nb = MultinomialNB()
nb.fit(train_features, train_labels)

# Make predictions on the validation and test sets
val_predictions = nb.predict(val_features)
test_predictions = nb.predict(test_features)

# Evaluate model performance on validation and test sets
val_accuracy = accuracy_score(val_labels, val_predictions)
test_accuracy = accuracy_score(test_labels, test_predictions)

print("Validation Accuracy (Naive Bayes): {:.2f}%".format(val_accuracy * 100))
print("Testing Accuracy (Naive Bayes): {:.2f}%".format(test_accuracy * 100))

Validation Accuracy (Naive Bayes): 63.29%
Testing Accuracy (Naive Bayes): 63.33%


In [ ]:
from sklearn.metrics import classification_report

# Generate classification report for validation set
val_report = classification_report(val_labels, val_predictions)
print("Classification Report (Validation Set):\n", val_report)

# Generate classification report for test set
test_report = classification_report(test_labels, test_predictions)
print("Classification Report (Test Set):\n", test_report)

Classification Report (Validation Set):
               precision    recall  f1-score   support

       anger       0.80      0.43      0.56      1096
        fear       0.79      0.54      0.64      1174
         joy       0.62      0.80      0.70      2645
        love       0.76      0.31      0.44       831
     neutral       0.60      0.01      0.02       351
     sadness       0.57      0.77      0.66      2587

    accuracy                           0.63      8684
   macro avg       0.69      0.48      0.50      8684
weighted avg       0.66      0.63      0.61      8684

Classification Report (Test Set):
               precision    recall  f1-score   support

       anger       0.77      0.43      0.55      1264
        fear       0.77      0.53      0.63      1475
         joy       0.62      0.80      0.70      3301
        love       0.79      0.31      0.44      1043
     neutral       0.50      0.01      0.01       455
     sadness       0.58      0.77      0.66      3317

 

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Load data
data = df

# Split data into training, validation, and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(data['Clean_Text'], data['Emotion'], test_size=0.2, random_state=42)
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

# Vectorize text data using CountVectorizer
vectorizer = CountVectorizer()
train_features = vectorizer.fit_transform(train_data)
val_features = vectorizer.transform(val_data)
test_features = vectorizer.transform(test_data)

# Train a Logistic Regression model
lr = LogisticRegression(max_iter=1000)
lr.fit(train_features, train_labels)

# Make predictions on the validation and test sets
val_predictions = lr.predict(val_features)
test_predictions = lr.predict(test_features)

# Evaluate model performance on validation and test sets
val_accuracy = accuracy_score(val_labels, val_predictions)
test_accuracy = accuracy_score(test_labels, test_predictions)

print("Validation Accuracy: {:.2f}%".format(val_accuracy * 100))
print("Testing Accuracy: {:.2f}%".format(test_accuracy * 100))

Validation Accuracy: 70.65%
Testing Accuracy: 70.50%


In [ ]:
from sklearn.metrics import classification_report

# Generate classification report for validation set
val_report = classification_report(val_labels, val_predictions)
print("Classification Report (Validation Set):\n", val_report)

# Generate classification report for test set
test_report = classification_report(test_labels, test_predictions)
print("Classification Report (Test Set):\n", test_report)

Classification Report (Validation Set):
               precision    recall  f1-score   support

       anger       0.77      0.60      0.67      1096
        fear       0.79      0.69      0.74      1174
         joy       0.71      0.77      0.74      2645
        love       0.67      0.54      0.60       831
     neutral       0.54      0.71      0.61       351
     sadness       0.69      0.75      0.72      2587

    accuracy                           0.71      8684
   macro avg       0.70      0.68      0.68      8684
weighted avg       0.71      0.71      0.71      8684

Classification Report (Test Set):
               precision    recall  f1-score   support

       anger       0.73      0.61      0.67      1264
        fear       0.77      0.70      0.73      1475
         joy       0.70      0.78      0.74      3301
        love       0.68      0.51      0.58      1043
     neutral       0.58      0.72      0.64       455
     sadness       0.70      0.73      0.72      3317

 

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Load data
data = df

# Split data into training, validation, and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(data['Clean_Text'], data['Emotion'], test_size=0.2, random_state=42)
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

# Vectorize text data using CountVectorizer
vectorizer = CountVectorizer()
train_features = vectorizer.fit_transform(train_data)
val_features = vectorizer.transform(val_data)
test_features = vectorizer.transform(test_data)

# Train a Logistic Regression model with L1 regularization
lr_l1 = LogisticRegression(max_iter=1000, penalty='l1', solver='liblinear')
lr_l1.fit(train_features, train_labels)

# Make predictions on the validation and test sets
val_predictions = lr_l1.predict(val_features)
test_predictions = lr_l1.predict(test_features)

# Evaluate model performance on validation and test sets
val_accuracy = accuracy_score(val_labels, val_predictions)
test_accuracy = accuracy_score(test_labels, test_predictions)

print("Validation Accuracy (L1 regularization): {:.2f}%".format(val_accuracy * 100))
print("Testing Accuracy (L1 regularization): {:.2f}%".format(test_accuracy * 100))

Validation Accuracy (L1 regularization): 70.83%
Testing Accuracy (L1 regularization): 71.12%


In [ ]:
from sklearn.metrics import classification_report

# Generate classification report for validation set
val_report = classification_report(val_labels, val_predictions)
print("Classification Report (Validation Set):\n", val_report)

# Generate classification report for test set
test_report = classification_report(test_labels, test_predictions)
print("Classification Report (Test Set):\n", test_report)

Classification Report (Validation Set):
               precision    recall  f1-score   support

       anger       0.80      0.59      0.68      1096
        fear       0.81      0.67      0.74      1174
         joy       0.70      0.78      0.74      2645
        love       0.67      0.56      0.61       831
     neutral       0.59      0.60      0.59       351
     sadness       0.68      0.76      0.72      2587

    accuracy                           0.71      8684
   macro avg       0.71      0.66      0.68      8684
weighted avg       0.71      0.71      0.71      8684

Classification Report (Test Set):
               precision    recall  f1-score   support

       anger       0.77      0.59      0.67      1264
        fear       0.79      0.69      0.74      1475
         joy       0.70      0.79      0.74      3301
        love       0.68      0.53      0.60      1043
     neutral       0.63      0.63      0.63       455
     sadness       0.70      0.75      0.72      3317

 

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Load data
data = df

# Split data into training, validation, and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(data['Clean_Text'], data['Emotion'], test_size=0.2, random_state=42)
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

# Vectorize text data using CountVectorizer
vectorizer = CountVectorizer()
train_features = vectorizer.fit_transform(train_data)
val_features = vectorizer.transform(val_data)
test_features = vectorizer.transform(test_data)

# Train a SVM model
svm = SVC(kernel='rbf')
svm.fit(train_features, train_labels)

# Make predictions on the validation and test sets
val_predictions = svm.predict(val_features)
test_predictions = svm.predict(test_features)

# Evaluate model performance on validation and test sets
val_accuracy = accuracy_score(val_labels, val_predictions)
test_accuracy = accuracy_score(test_labels, test_predictions)

print("Validation Accuracy: {:.2f}%".format(val_accuracy * 100))
print("Testing Accuracy: {:.2f}%".format(test_accuracy * 100))

Validation Accuracy: 68.24%
Testing Accuracy: 68.45%


In [ ]:
from sklearn.metrics import classification_report

# Generate classification report for validation set
val_report = classification_report(val_labels, val_predictions)
print("Classification Report (Validation Set):\n", val_report)

# Generate classification report for test set
test_report = classification_report(test_labels, test_predictions)
print("Classification Report (Test Set):\n", test_report)

Classification Report (Validation Set):
               precision    recall  f1-score   support

       anger       0.89      0.46      0.61      1096
        fear       0.88      0.57      0.69      1174
         joy       0.65      0.81      0.72      2645
        love       0.73      0.42      0.53       831
     neutral       0.61      0.72      0.66       351
     sadness       0.63      0.78      0.69      2587

    accuracy                           0.68      8684
   macro avg       0.73      0.63      0.65      8684
weighted avg       0.71      0.68      0.68      8684

Classification Report (Test Set):
               precision    recall  f1-score   support

       anger       0.85      0.45      0.59      1264
        fear       0.88      0.61      0.72      1475
         joy       0.65      0.81      0.73      3301
        love       0.72      0.41      0.52      1043
     neutral       0.64      0.71      0.67       455
     sadness       0.64      0.76      0.70      3317

 

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Load data
data = df

# Split data into training, validation, and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(data['Clean_Text'], data['Emotion'], test_size=0.2, random_state=42)
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

# Vectorize text data using CountVectorizer
vectorizer = CountVectorizer()
train_features = vectorizer.fit_transform(train_data)
val_features = vectorizer.transform(val_data)
test_features = vectorizer.transform(test_data)

# Train a simple Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(train_features, train_labels)

# Make predictions on the validation and test sets
val_predictions = rf.predict(val_features)
test_predictions = rf.predict(test_features)

# Evaluate model performance on validation and test sets
val_accuracy = accuracy_score(val_labels, val_predictions)
test_accuracy = accuracy_score(test_labels, test_predictions)

print("Validation Accuracy (Random Forest): {:.2f}%".format(val_accuracy * 100))
print("Testing Accuracy (Random Forest): {:.2f}%".format(test_accuracy * 100))

Validation Accuracy (Random Forest): 67.07%
Testing Accuracy (Random Forest): 67.38%


In [ ]:
from sklearn.metrics import classification_report

# Generate classification report for validation set
val_report = classification_report(val_labels, val_predictions)
print("Classification Report (Validation Set):\n", val_report)

# Generate classification report for test set
test_report = classification_report(test_labels, test_predictions)
print("Classification Report (Test Set):\n", test_report)

Classification Report (Validation Set):
               precision    recall  f1-score   support

       anger       0.81      0.57      0.67      1096
        fear       0.87      0.63      0.73      1174
         joy       0.69      0.75      0.72      2645
        love       0.70      0.48      0.57       831
     neutral       0.29      0.81      0.42       351
     sadness       0.69      0.69      0.69      2587

    accuracy                           0.67      8684
   macro avg       0.67      0.65      0.63      8684
weighted avg       0.71      0.67      0.68      8684

Classification Report (Test Set):
               precision    recall  f1-score   support

       anger       0.73      0.55      0.63      1264
        fear       0.87      0.67      0.76      1475
         joy       0.69      0.76      0.72      3301
        love       0.71      0.45      0.55      1043
     neutral       0.28      0.79      0.42       455
     sadness       0.72      0.69      0.70      3317

 

In [ ]:
def rec(input):
  input = vectorizer.transform([input])
  input = input.toarray()
  input = input.flatten()
  pre = lr.predict([input])
  print(f'Result:',pre[0])
  recommeded_songs = get_songs_by_emotion(pre[0])
  print(f'Songs:',recommeded_songs)
  return recommeded_songs


In [ ]:
# import random

# data = pd.read_csv("/content/drive/MyDrive/ML datasets/Cleaned_Bollywood_dataset.csv")

# def get_songs_by_emotion(emotion):
#     song_list = []
#     for i in data.index:
#         if data.loc[i, "emotion"] == emotion:
#             song_list.append(data.loc[i, "Song-Name"])
#     return random.sample(song_list,3)

In [ ]:
import json
import random

data = pd.read_csv("/content/drive/MyDrive/ML datasets/Cleaned_Bollywood_dataset.csv")

def get_songs_by_emotion(emotion):
    song_list = []
    for i in data.index:
        if data.loc[i, "emotion"] == emotion:
            song_list.append({
                "videoId": data.loc[i, "Song-Name"],
                "title": data.loc[i, "Song-Name"],
                "thumbnailUrl": data.loc[i, "Song-Name"],
                "channelTitle": data.loc[i, "Song-Name"]
            })
    return json.dumps(random.sample(song_list,3))


In [ ]:
string = "My life is messed up"
songs = rec(string)
print(f'Statement:',string)
songs

Result: sadness
Songs: [{"videoId": "Tujhse Naaraz Nahi Zindagi", "title": "Tujhse Naaraz Nahi Zindagi", "thumbnailUrl": "Tujhse Naaraz Nahi Zindagi", "channelTitle": "Tujhse Naaraz Nahi Zindagi"}, {"videoId": "Jay-Jaykara", "title": "Jay-Jaykara", "thumbnailUrl": "Jay-Jaykara", "channelTitle": "Jay-Jaykara"}, {"videoId": "Yeh Kasoor Mera Hai", "title": "Yeh Kasoor Mera Hai", "thumbnailUrl": "Yeh Kasoor Mera Hai", "channelTitle": "Yeh Kasoor Mera Hai"}]
Statement: My life is messed up


'[{"videoId": "Tujhse Naaraz Nahi Zindagi", "title": "Tujhse Naaraz Nahi Zindagi", "thumbnailUrl": "Tujhse Naaraz Nahi Zindagi", "channelTitle": "Tujhse Naaraz Nahi Zindagi"}, {"videoId": "Jay-Jaykara", "title": "Jay-Jaykara", "thumbnailUrl": "Jay-Jaykara", "channelTitle": "Jay-Jaykara"}, {"videoId": "Yeh Kasoor Mera Hai", "title": "Yeh Kasoor Mera Hai", "thumbnailUrl": "Yeh Kasoor Mera Hai", "channelTitle": "Yeh Kasoor Mera Hai"}]'

In [ ]:
# from sklearn.pipeline import Pipeline
# from sklearn.svm import SVC
# from sklearn.feature_extraction.text import CountVectorizer

# pipe_svm = Pipeline(steps=[('cv', CountVectorizer()), ('svm', SVC())])
# pipe_svm.fit(x_train, y_train)

# score = pipe_svm.score(x_test, y_test)
# print(f'Accuracy score: {score:.4f}')


In [ ]:
# from sklearn.metrics import classification_report

# y_pred = pipe_svm.predict(x_test)
# report = classification_report(y_test, y_pred)
# print(report)

In [ ]:
# from sklearn.pipeline import Pipeline
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.feature_extraction.text import CountVectorizer

# pipe_rf = Pipeline(steps=[('cv', CountVectorizer()), ('rf', RandomForestClassifier())])
# pipe_rf.fit(x_train, y_train)

# score = pipe_rf.score(x_test, y_test)
# print(f'Accuracy score: {score:.4f}') 


In [ ]:
# from sklearn.metrics import classification_report

# y_pred = pipe_rf.predict(x_test)
# report = classification_report(y_test, y_pred)
# print(report)

In [ ]:
test_features

<7043x25295 sparse matrix of type '<class 'numpy.int64'>'
	with 51496 stored elements in Compressed Sparse Row format>

In [ ]:
import pickle

pickle_out = open('classifier.pkl',"wb")
pickle.dump(lr,pickle_out)
pickle_out.close()